In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)
driver.get("https://www.amf-france.org/fr/actualites-publications/la-une/toutes-les-actualites-et-publications")

df = pd.DataFrame(columns=['Date', 'Titre', 'Contenu', 'URL'])


# Attendre que la table soit visible
full_xpath = "/html/body/div[2]/div[1]/main/div[3]/div/div[2]/div/div[2]/div/div[2]/div/div[2]/table"
WebDriverWait(driver, 30).until(
    EC.visibility_of_element_located((By.XPATH, full_xpath))
)

rows = driver.find_elements(By.XPATH, f"{full_xpath}/tbody/tr")
urls = [row.find_element(By.XPATH, ".//td[3]/a").get_attribute('href') for row in rows]


# Traiter chaque URL
for url in urls:
    print(f"Traitement de l'URL : {url}")
    driver.get(url)
    article = {'URL': url}

    try:
        article['Titre'] = driver.find_element(By.CSS_SELECTOR, '.like-h1').text
    except:
        article['Titre'] = "inconnu"

    try:
        article['Date'] = driver.find_element(By.CSS_SELECTOR, '.date').text
    except:
        article['Date'] = "inconnu"

    try:
        div = driver.find_element(By.CSS_SELECTOR, ".paragraph.paragraph--type--wysiwyg.paragraph--view-mode--default")
        paragraphs = div.find_elements(By.TAG_NAME, "p")
        article['Contenu'] = "\n".join([p.text for p in paragraphs if len(p.text) >= 100])
    except:
        article['Contenu'] = "non trouvé"

        # Ajouter au DataFrame si l'URL n'est pas déjà présente
    if not df[df['URL'] == url].any(axis=None):
        df = pd.concat([df, pd.DataFrame([article])], ignore_index=True)

driver.quit()
display(df)

Traitement de l'URL : https://www.amf-france.org/fr/le-mediateur/journal-de-bord-du-mediateur/dossiers-du-mois/labsence-de-reponse-un-questionnaire-de-connaissance-client-des-consequences-distinctes-selon-quil
Traitement de l'URL : https://www.amf-france.org/fr/actualites-publications/prises-de-parole/discours-de-cloture-de-marie-anne-barbat-layani-presidente-de-lamf-les-20-ans-de-lautorite-des
Traitement de l'URL : https://www.amf-france.org/fr/actualites-publications/actualites/lamf-publie-une-etude-sur-la-relation-flux-performance-des-fonds-obligataires-francais
Traitement de l'URL : https://www.amf-france.org/fr/actualites-publications/publications/rapports-etudes-et-analyses/relation-flux-performance-des-fonds-obligataires-francais
Traitement de l'URL : https://www.amf-france.org/fr/actualites-publications/communiques/communiques-de-lamf/la-commission-des-sanctions-de-lamf-sanctionne-la-societe-visiomed-et-ses-anciens-dirigeants-mm-eric
Traitement de l'URL : https://www.amf-france

,Date,Titre,Contenu,URL
0,04 décembre 2023,L'absence de réponse à un questionnaire de con...,"Monsieur L, détenteur d’un compte-titres dans ...",https://www.amf-france.org/fr/le-mediateur/jou...
1,01 décembre 2023,Discours de clôture de Marie-Anne Barbat-Layan...,"Je voudrais, en premier lieu, remercier très s...",https://www.amf-france.org/fr/actualites-publi...
2,01 décembre 2023,L'AMF publie une étude sur la relation flux-pe...,Les fonds obligataires détiennent principaleme...,https://www.amf-france.org/fr/actualites-publi...
3,01 décembre 2023,Relation flux performance des fonds obligatair...,non trouvé,https://www.amf-france.org/fr/actualites-publi...
4,29 novembre 2023,La Commission des sanctions de l’AMF sanctionn...,La Commission des sanctions a d’abord écarté l...,https://www.amf-france.org/fr/actualites-publi...
5,28 novembre 2023,Décision du 31 octobre 2023 relative à la modi...,non trouvé,https://www.amf-france.org/fr/reglementation/r...
6,27 novembre 2023,Le numérique et l’évolution rapide des technol...,"Lors de ces échanges, les représentants de l’A...",https://www.amf-france.org/fr/actualites-publi...
7,23 novembre 2023,L'AMF a 20 ans,Les 20 ans de l’AMF sont l’occasion de faire l...,https://www.amf-france.org/fr/actualites-publi...
8,22 novembre 2023,La « gamification » tend à augmenter la prise ...,Le Laboratoire d’économie expérimentale de Str...,https://www.amf-france.org/fr/actualites-publi...
9,22 novembre 2023,Gamification et copy trading en finance,non trouvé,https://www.amf-france.org/fr/gamification-et-...
